# Self-Driving Car Engineer Nanodegree

## Project: **Advanced Finding Lane Lines on the Road** 

***
In this project, advanced computer vision technologies are applied to find lane lines and identify the drivable lane area ahead.

The project first calibrates the camera based on the calibration pattern images, then it goes through following steps for finding lane lines:
1. Correct distortion for image/video frame
2. Derive binary image based on the thresholds combination of color space and gradients
3. Convert to birds-eye's view through perspective transform
4. Detect lane lines
   a. Use histogram and sliding window to find pixels belongs to lane lines
   b. Or search lane lines based on the line positions in previous video frame 
5. Draw lane line and drivable lane area
6. Calculate and check lane information (curvature, vehicle position relative to lane center, lane width)
7. Convert back to original perspective
8. Visualize the image/video

# Import Packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

# Camera Calibration
In this section, the parameters for calibrating the camera used in this project are calculated based on a set of calibration images. The calibration parameters include: camera matrix, distortion coefficients, camera position vectors (rotation and translation).

A class (Cam) is defined for storing the calibration parameters for use later.

In [ ]:
class Cam:
    '''
    Class for storing the calibration parameters of a camera.
    It stores: camera matrix (mtx), distortion coefficients (dist)，
               rotation vector (rvecs), translation vector (tvecs).
    '''
    def __init__(self):
        self.mtx = None
        self.dist = None
        self.rvecs = None
        self.tvecs = None
    
    def set_calibration_param(self, mtx, dist, rvecs, tvecs):
        self.mtx = mtx
        self.dist = dist
        self.rvecs = rvecs
        self.tvecs = tvecs

def calibrate_camera(images, cols, rows):
    '''
    This function calculates the calibration parameters for a camera.
    images:  a set of chessboard pattern images, which are taken in different scenarios.
    cols:  Number of inside corners in each row in chessboard pattern images
    rows:  Number of inside corners in each column in chessboard pattern images
    '''
    objpoints = []   # Points in real world (3D, z = 0)
    imgpoints = []   # Points in image plane (2D)

    # Prepare object points (same for all images)
    objp = np.zeros((rows*cols, 3), np.float32)
    objp[:,:2] = np.mgrid[0:cols,0:rows].T.reshape(-1,2)

    gray = None

    for image in images:
        img = mpimg.imread(image)
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (cols, rows), None)
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    # Calculate calibration and distortion coefficients
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None) 
    return ret, mtx, dist, rvecs, tvecs


print("Calibrating Frame...")
images = glob.glob('./camera_cal/calibration*.jpg')
ret, mtx, dist, rvecs, tvecs = calibrate_camera(images, 9, 6)

# Save the calibration parameters in cam, which could be used later for correcting images and video frames.
cam = Cam()
cam.set_calibration_param(mtx, dist, rvecs, tvecs)
print("Calibrating Frame...Done!")

In [ ]:
def correct_image(img, cam):
    '''
    Undistort the image based on the calibration parameter stored in cam
    '''
    undist_img = cv2.undistort(img, cam.mtx, cam.dist, None, cam.mtx)
    return undist_img

# Undistort Image/Video Frames
Image and video frames should be undistorted before further steps for finding lanes.


In [ ]:
# Test image correction with an example calibration image
calibration_image = './camera_cal/calibration1.jpg'
cali_img = mpimg.imread(calibration_image)
undist_cali_img = correct_image(cali_img, cam)
undist_cali_image = './output_images/calibration1_undist.jpg'
mpimg.imsave(undist_cali_image, undist_cali_img)

fig = plt.figure(figsize=(20, 10))
fig.add_subplot(1, 2, 1)
plt.imshow(cali_img)
plt.title('Original', fontsize=22)

fig.add_subplot(1, 2, 2)
plt.imshow(undist_cali_img)
plt.title('Undistorted', fontsize=22)

# Test image correction on test images
test_images = glob.glob('./test_images/test*.jpg')
tmp_images = glob.glob('./test_images/straight_lines*.jpg')
test_images = test_images + tmp_images

rows = len(test_images)

dst_dir = './output_images/'
for image_file in test_images:
    (path_filename, ext) = os.path.splitext(image_file)
    filename = os.path.basename(path_filename)
    undist_image_file = dst_dir + filename + "_undist" + ext
    img = mpimg.imread(image_file)
    undist_img = correct_image(img, cam)
    mpimg.imsave(undist_image_file, undist_img)
    
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    ax1.imshow(img)
    ax1.set_title('Original', fontsize=22)
    ax2.imshow(undist_img)
    ax2.set_title('Undistorted', fontsize=22)
    
print('Undistorting images test done!')

# Color and Gradient Thresholding
In order to identify the lane lines, the pixels belonging to the lane lines should be detected.
To do that, the masks in different color spaces (e.g. RGB, HLS, HSV, Lab, LUV), and also gradient change in different directions and perspectives are defined (e.g. gradient in x-axis, y-axis, magnitude, x-y direction ratio) and combined for extracting the lane pixels. 

In [ ]:
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    '''
    Gradient thresholding with Sobel operator along x or y axis
    '''
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take the derivative in x or y given orient = 'x' or 'y'
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)

    # Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    #print(abs_sobel.shape)
    # Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    # Create a mask of 1's where the scaled gradient magnitude
    # is > thresh_min and < thresh_max
    sbinary = np.zeros_like(scaled_sobel)
    # Return this mask as your binary_output image
    sbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return sbinary


def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    '''
    Gradient thresholding with Sobel operator with magnitude
    '''
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take the gradient in x and y separately
    sobelX = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobelY = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)
    # Calculate the magnitude
    mag_sobel = np.sqrt(sobelX**2 + sobelY**2)
    # Scale to 8-bit (0 - 255) and convert to type = np.uint8
    scaled_sobel = np.uint8(255 * mag_sobel / np.max(mag_sobel))
    # Create a binary mask where mag thresholds are met
    sbinary = np.zeros_like(scaled_sobel)
    # Return this mask as your binary_output image
    sbinary[(scaled_sobel >= mag_thresh[0]) & (scaled_sobel <= mag_thresh[1])] = 1
    return sbinary


def dir_thresh(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    '''
    Gradient thresholding based on direction
    '''
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Take the gradient in x and y separately
    sobelX = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize = sobel_kernel)
    sobelY = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize = sobel_kernel)

    # Take the absolute value of the x and y gradients
    abs_sobelX = np.absolute(sobelX)
    abs_sobelY = np.absolute(sobelY)

    # Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient
    direction_sobel = np.arctan2(abs_sobelY, abs_sobelX)
    # Create a binary mask where direction thresholds are met
    sbinary = np.zeros_like(direction_sobel)
    # Return this mask as your binary_output image
    sbinary[(direction_sobel >= thresh[0]) & (direction_sobel <= thresh[1])] = 1
    return sbinary


def color_hls_thresh(img, s_thresh=(90, 255)):
    # Convert to HLS color space and separate the S channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:, :, 2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    return s_binary


def color_luv_thresh(img, l_thresh=(225, 255)):
    # Convert to Luv color space and separate the L channel
    luv = cv2.cvtColor(img, cv2.COLOR_RGB2Luv)
    l_channel = luv[:,:,0]
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    return l_binary


def color_lab_thresh(img, b_thresh=(155, 200)):
    # Convert to Lab color space and separate the b channel
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
    b_channel = lab[:,:,2]
    b_binary = np.zeros_like(b_channel)
    b_binary[(b_channel >= b_thresh[0]) & (b_channel <= b_thresh[1])] = 1
    return b_binary
 
    
def color_gradient_threshold(img):
    '''
    This function create binary image by thresholding in color space and/or gradient.
    '''
    # HLS space
    s_binary = color_hls_thresh(img, s_thresh=(170, 255))
    
    # LUV space
    l_binary = color_luv_thresh(img, l_thresh=(225, 255))
    
    # Lab space
    b_binary = color_lab_thresh(img, b_thresh=(155, 200))

    # HSV space
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    yellow_min = np.array([20,60,60])
    yellow_max = np.array([38,174,250])
    yellow = cv2.inRange(hsv, yellow_min, yellow_max)
    
    # RGB space
    white_min = np.array([200,200,200])
    white_max = np.array([255,255,255])
    white = cv2.inRange(img, white_min, white_max)

    # Sobel gradient
    ksize = 3
    gradx = abs_sobel_thresh(img, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    grady = abs_sobel_thresh(img, orient='y', sobel_kernel=ksize, thresh=(20, 100))
    mag_binary = mag_thresh(img, sobel_kernel=ksize, mag_thresh=(30, 100))
    dir_binary = dir_thresh(img, sobel_kernel=ksize, thresh=(0.7, 1.3))
    
    # Stack each channel
    color_img = np.dstack((np.zeros_like(gradx), gradx, s_binary)) * 255
    #combined_binary = np.zeros_like(gradx)
    #combined_binary[(s_binary == 1) | (gradx == 1)] = 1

    combined_binary = np.zeros_like(s_binary)
    #combined_binary[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1)) | (s_binary == 1)] = 1

    combined_binary[(yellow >= 1) | (white >=1) | (l_binary == 1) | (b_binary == 1)] = 1
    ##combined_binary[(yellow >= 1) | (white >=1) | (l_binary == 1) | (b_binary == 1) | ((mag_binary == 1) & (dir_binary == 1))] = 1

    return color_img, combined_binary

In [ ]:
# Create thresholded binary image based on color transform and gradients
for image_file in test_images:
    (path_filename, ext) = os.path.splitext(image_file)
    filename = os.path.basename(path_filename)
    combined_thresholded_image_file = dst_dir + filename + "_combined_thresholded" + ext
    img = mpimg.imread(image_file)
    undist_img = correct_image(img, cam)
    color_img, combined_thresholded_img = color_gradient_threshold(undist_img)
    mpimg.imsave(combined_thresholded_image_file, combined_thresholded_img*255, cmap = 'gray')
    
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    ax1.imshow(undist_img)
    ax1.set_title('Undistorted', fontsize=22)
    ax2.imshow(combined_thresholded_img, cmap = 'gray')
    ax2.set_title('Color & Binary Thresholded Binary', fontsize=22)

# Perspective Transform
After creating color and/or gradient thresholded binary image, perspective transform is carried out to convert to a birds-eye view, which allows easily find the curvature of the lanes.

In order to perform perspective transform, positions of four points in original image and converted image are defined as follows:

Source Points: [594,447], [690,447], [1130,720], [195,720]
Target Points: [310,0], [960,0], [960,680], [310,680]

Note: After identifying the lane lines intthe birds-eye view image, at the end, the image needs to be converted back (i.e. inverse perspective transform) to original perspective for overlaying the lane lines on the raw image/video frame. 

In [ ]:
def perspective_transform(img, inverse = False):
    '''
    Transform the image to birds-eye perspective (inverse=False), 
    or convert from birds-eye perspective to original perspective
    
    img: undistorted image to be transformed perspective
    '''
    # src: 4 source points in original image
    src = np.float32([[594,447], [690,447], [1130,720], [195,720]])
    # dst: points in target image
    dst = np.float32([[310,0], [960,0], [960,680], [310,680]])
    # use cv2.getPerspectiveTransform() to get M, the transform matrix
    if inverse == False:
        M = cv2.getPerspectiveTransform(src, dst)
    else:
        M = cv2.getPerspectiveTransform(dst, src)
    # use cv2.warpPerspective() to warp your image to a top-down view
    transformed = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)

    return transformed, M

In [ ]:
# Test perspective transform on test images
for image_file in test_images:
    (path_filename, ext) = os.path.splitext(image_file)
    filename = os.path.basename(path_filename)
    transformed_combined_thresholded_image = dst_dir + filename + "_transformed_combined_thresholded" + ext
    transformed_color_image = dst_dir + filename + "_transformed_color" + ext
    img = mpimg.imread(image_file)
    undist_img = correct_image(img, cam)
    color_img, combined_thresholded_img = color_gradient_threshold(undist_img)
    
    transformed_combined_thresholded_img, M = perspective_transform(combined_thresholded_img)
    transformed_color_img, M = perspective_transform(undist_img)
    
    mpimg.imsave(transformed_combined_thresholded_image, transformed_combined_thresholded_img*255, cmap = 'gray')
    mpimg.imsave(transformed_color_image, transformed_color_img)
    
    _, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
    ax1.imshow(undist_img)
    ax1.set_title('Original (corrected)')
    ax2.imshow(transformed_combined_thresholded_img, cmap = 'gray')
    ax2.set_title('Perspective Transformed (Binary)')
    ax3.imshow(transformed_color_img)
    ax3.set_title('Perspective Transformed')

# Lane Line Pixel Detection
To find the pixels belonging to the lane lines, a histogram is calculated along the y axis of the warped binary image.
The histogram shows two peaks, which corresponds the base positions of the two lane lines at the bottom of the image.
From these base positions, sliding windows are defined and used to search for the pixels for the lane lines towards the top of the images.
The searching process keeps updating the base positions for the sliding windows based on the distribution of the lane line pixels in the previous window.
The function find_lane_pixels() carries out the lane pixel searching.

In [ ]:
def find_lane_pixels(binary_warped):
    '''
    Function for searching pixels belonging to lane lines.
    '''
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        # Find the four below boundaries of the window ###
        win_xleft_low = max(0, leftx_current - margin)
        win_xleft_high = min(binary_warped.shape[1], leftx_current + margin)
        win_xright_low = max(0, rightx_current - margin)
        win_xright_high = min(binary_warped.shape[1], rightx_current + margin)
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
                              (win_xleft_high,win_y_high),(255,255,0), 3) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
                              (win_xright_high,win_y_high),(255,255,0), 3) 
        
        # Identify the nonzero pixels in x and y within the window
        leftWin = (nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)
        good_left_inds = leftWin.nonzero()[0]
        rightWin = (nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)
        good_right_inds = rightWin.nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If found > minpix pixels, recenter next window
        # (`right` or `leftx_current`) on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))

        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [0, 255, 0]
    out_img[righty, rightx] = [0, 0, 255]

    return leftx, lefty, rightx, righty, histogram, out_img

In [ ]:
# Test for lane pixel searching
test_image = './test_images/straight_lines1.jpg'
img = mpimg.imread(test_image)
undist_img = correct_image(img, cam)
color_img, combined_thresholded_img = color_gradient_threshold(undist_img)
transformed_combined_thresholded_img, M = perspective_transform(combined_thresholded_img)
leftx, lefty, rightx, righty, histogram, sliding_win_img = find_lane_pixels(transformed_combined_thresholded_img)

sliding_win_image = './output_images/straight_lines1_slidingwin.jpg'
mpimg.imsave(sliding_win_image, sliding_win_img)

histogram_image = './output_images/straight_lines1_histogram.jpg'

plt.plot(histogram)
plt.title("Histogram of Pixels", fontsize=22)
plt.savefig(histogram_image)

_, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
ax1.imshow(transformed_combined_thresholded_img, cmap = 'gray')
ax1.set_title('Perspective Transformed (Binary)', fontsize=22)
ax2.imshow(sliding_win_img)
ax2.set_title('Sliding Windows', fontsize=22)

# Fit Polynomial Lines
Once find the pixels belonging to the lane lines, we can fit polynomial lines to the pixels for drawing the lane lines later.

The following function search_lane_raw() fits two polynomial lines (i.e. the left and right lane lies) to the pixels detected in previous steps. It returns the coefficients of the polynomial lines as expression f(y) = A*y^2 + B*y + C

In [ ]:
def search_sliding_window(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, histogram, out_img = find_lane_pixels(binary_warped)
    if (len(lefty) < 20) or (len(righty) < 20):
        return False, None, None, None
    
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    return True, left_fit, right_fit, out_img

Once we have the coefficients of the polynomial lines, we could use following function cal_left_right_fitx() to derive positions of the pixels forming the lane lines.

The function draw_lane() calls cal_left_right_fitx() to obtains pixels of fitted lane lines, and then draw the lane lines and drivable lane area.

In [ ]:
def cal_left_right_fitx(img, left_fit, right_fit):
    # Generate x and y values for plotting
    ploty = np.linspace(0, img.shape[0]-1, img.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        
    return left_fitx, right_fitx, ploty


def draw_lane(binary_warped, left_fit, right_fit):
    # Draw lane lines and drivable lane area
    left_fitx, right_fitx, ploty = cal_left_right_fitx(binary_warped, left_fit, right_fit)

    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    window_img = np.zeros_like(out_img)
    left_line_win = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    right_line_win = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    lane_area_pts = np.hstack((left_line_win, right_line_win))
    cv2.polylines(window_img, np.int_([lane_area_pts]), isClosed=False, color=(255,0,0), thickness=20)
    cv2.fillPoly(window_img, np.int_([lane_area_pts]), (0,255,0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    return window_img, left_fitx, right_fitx, ploty

In [ ]:
# Test drawing lane lines and drivable lane area
test_image = './test_images/straight_lines1.jpg'
img = mpimg.imread(test_image)
undist_img = correct_image(img, cam)
color_img, combined_thresholded_img = color_gradient_threshold(undist_img)
transformed_combined_thresholded_img, M = perspective_transform(combined_thresholded_img)
found, left_fit, right_fit, sliding_win_img = search_sliding_window(transformed_combined_thresholded_img)
lane_img, left_fitx, right_fitx, ploty = draw_lane(transformed_combined_thresholded_img, left_fit, right_fit)

left_line_win = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
right_line_win = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
lane_area_pts = np.hstack((left_line_win, right_line_win))
cv2.polylines(sliding_win_img, np.int_([lane_area_pts]), isClosed=False, color=(255,0,0), thickness=5)

lane_area_image = './output_images/straight_lines1_drivable_area.jpg'
mpimg.imsave(lane_area_image, lane_img)

line_fitted_image = './output_images/straight_lines1_fitted_line.jpg'
mpimg.imsave(line_fitted_image, sliding_win_img)

_, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
ax1.imshow(transformed_combined_thresholded_img, cmap = 'gray')
ax1.set_title('Perspective Transformed (Binary)', fontsize=22)
ax2.imshow(sliding_win_img)
ax2.set_title('Lane Lines Fitted', fontsize=22)
ax3.imshow(lane_img)
ax3.set_title('Lane Lines and Drivable Lane Area', fontsize=22)

# Lane Curvature and Vehicle Position
After identifying the lane lines and drivable area, we can calculate the radius of the curvature of the lane.

    Radius = (1+(2*A*y + B)^2)^(3/2)/abs(2*A)

In the formula, A, B are the coefficients of the polynomial line corresponding to the lane line. y is the base position of the lane line at the bottom of the image. 

Based on the base positions of the two lane lines, we can estimate the offset of the vehicle from the lane center.
The center of the lane is the center of the two base positions of the lane lines at the bottom edge. The vehicle's center position is at the center of the image's x axis. The difference of these lane center and the vehicle center is the offset of the vehicle from the lane center.

So far, all the calculation is based on pixel. To make it practical, we need to calculate the lane curvature and vehicle position in practical units.
The relationship between pixel and real length is defined as follows:

 - 720 pixels in y axis equal to 30 meters in reality
 - 700 pixels in x axis equal to 3.7 meters (which is the width of a lane)

In [ ]:
def measure_curvature_real(leftx, rightx, ploty):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
    rightx = rightx[::-1]  # Reverse to match top-to-bottom in y

    # Fit a second order polynomial to pixel positions in lane line
    left_fit_cr = np.polyfit(ploty * ym_per_pix, leftx * xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty * ym_per_pix, rightx * xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty) * ym_per_pix
    
    # calculation of R_curve (radius of curvature)
    left_curverad = (1 + (2*left_fit_cr[0]*y_eval + left_fit_cr[1])**2)**1.5/np.absolute(2*left_fit_cr[0])
    right_curverad = (1 + (2*right_fit_cr[0]*y_eval + right_fit_cr[1])**2)**1.5/np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad


def cal_car_pos(img, left_fit, right_fit):
    '''
    Calculate the offset of the vehicle from the lane center,
    and also the detected lane width
    '''
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    leftx = left_fit[0]*img.shape[0]**2 + left_fit[1]*img.shape[0] + left_fit[2]
    rightx = right_fit[0]*img.shape[0]**2 + right_fit[1]*img.shape[0] + right_fit[2]
    car_pos_offset = (img.shape[1]/2 - (rightx + leftx)/2) * xm_per_pix
    lane_width = (rightx - leftx)*xm_per_pix
    
    return car_pos_offset, lane_width

# Sanity Check
For each lane line detection, we need to check whether the detection is valid.
In this project, we go through sanity check for the calculated radius of lane curvature, vehicle offset from lane center and lane width.
If the detected lane lines are valid, they should have similar curvature; the vehicle should be in certain offset from the lane center and the lane width should be a valid value.

The following function does the sanity check.

In [ ]:
def is_valid_lane(left_curverad, right_curverad, car_offset, lane_width):
    '''
    Sanity check for the detected lane curvature, vehicle offset and lane width.
    '''
    ratio = left_curverad / right_curverad
    if (ratio > 5) or (ratio < 0.2):
        return False

    if abs(car_offset) > 1.0:
        return False
    
    if (lane_width < 2) or (lane_width > 4.4):
        return False
    
    return True

# Transform Back to Original Perspective
Once we have a valid detection of the lane lines and have drawn the drivable area (as shown in previous sections), we need to convert the image back to the original perspective for visualization.
This is done through inverse perspective transform as illustrated below. 

The code below also shows the final image/video frame with the identified lane lines and drivable area overlaid on the original image/video frame.

In [ ]:
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """ img is the output of the hough_lines(), An image with lines drawn on it. 
    Should be a blank image (all black) with lines drawn on it.
    `initial_img` should be the image before any processing.

    The result image is computed as follows:
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)


# Convert back to original perspective
test_image = './test_images/straight_lines1.jpg'
img = mpimg.imread(test_image)
undist_img = correct_image(img, cam)
color_img, combined_thresholded_img = color_gradient_threshold(undist_img)
transformed_combined_thresholded_img, M = perspective_transform(combined_thresholded_img)
found, left_fit, right_fit, sliding_win_img = search_sliding_window(transformed_combined_thresholded_img)
lane_img, left_fitx, right_fitx, ploty = draw_lane(transformed_combined_thresholded_img, left_fit, right_fit)

inv_transformed_lane_img, M = perspective_transform(lane_img, inverse=True)
result_img = weighted_img(inv_transformed_lane_img, img, 1, 0.4, 0)

inv_lane_area_image = './output_images/straight_lines1_inv_transformed.jpg'
mpimg.imsave(inv_lane_area_image, inv_transformed_lane_img)

overlaid_lane_area_image = './output_images/straight_lines1_lane_overlaid.jpg'
mpimg.imsave(overlaid_lane_area_image, result_img)

_, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
ax1.imshow(inv_transformed_lane_img)
ax1.set_title('Original Perspective', fontsize=22)
ax2.imshow(result_img)
ax2.set_title('Lane Line and Drivable Area Overlaid', fontsize=22)

# Optimization for Lane Finding in Video
When we search lane lines in a series of video frames, the positions of the lane lines from one frame to the next will not change significantly. (Camera usually streams frames at a rate 24+ fps.)
Once we find the lane lines in a frame, in the next frame, we don't need to search blindly with sliding windows as implemented in search_sliding_window(). Instead, we can search the line in the new frame based on the information of the previously fitted polynomial lines in previous frame, which is done in function search_around_poly() defined below.

In [ ]:
def search_around_poly(binary_warped, left_fit, right_fit, margin = 100):
    '''
    Search polynomial lines in the frame based on line position in previous frame(s).
    left_fit, right_fit: The polynomial coefficients of the lane lines in previous frame(s).
    '''
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Set the area of search based on activated x-values
    # within the +/- margin of our polynomial function
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) &
                      (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) &
                       (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    if (len(lefty) < 20) or (len(righty) < 20):
        return False, None, None

    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    return True, left_fit, right_fit

To smooth the lane line detection and drawing, a LaneLine class is defined, which is used to keep tracking the lane line information in recent frames. 
The parameters of lane lines to be drawn in current frame are averaged over the parameters in recent frames.

In [ ]:
class LaneLine():
    '''
    This class keeps tracking the lane lines identified in the recent frames.
    '''
    def __init__(self):
        # was the line detected in the last iteration?
        self.last_detected = False  
        # Coefficients of left/right polynomial lines in recent frames
        self.recent_fit_left = []
        self.recent_fit_right = []
        # Current coefficient of left/right polynomial line to be used (for searching in next frame)
        self.best_fit_left = None
        self.best_fit_right = None
        # Number of continuous frames failed to be found lane lines
        # If continuously failed certain times, switch back to raw sliding window search
        self.cont_bad_frames = 0
        # Total frame count
        self.frame_cnt = 0
        # Threshold for recent_fit_left
        self.iterative_win = 3
        # Threshold for continuous bad frames
        self.bad_frames_threshold = 3
        
    def update(self, left_fit, right_fit):
        self.frame_cnt = self.frame_cnt + 1
        if (left_fit is None) or (right_fit is None):
            self.last_detected = False
            self.cont_bad_frames = self.cont_bad_frames + 1
            if self.cont_bad_frames >= self.bad_frames_threshold:
                self.recent_fit_left = []
                self.recent_fit_right = []
                self.best_fit_left = None
                self.best_fit_right = None
        else:
            self.last_detected = True
            self.cont_bad_frames = 0
            self.recent_fit_left.append(left_fit)
            self.recent_fit_right.append(right_fit)
            if len(self.recent_fit_left) > self.iterative_win:
                self.recent_fit_left = self.recent_fit_left[1:]
                self.recent_fit_right = self.recent_fit_right[1:]
            if len(self.recent_fit_left) > 0:
                self.best_fit_left = np.mean(self.recent_fit_left, axis=0)
                self.best_fit_right = np.mean(self.recent_fit_right, axis=0)

# Pipeline
The whole pipeline for identifying lane and visualize the lane lines, drivable area, lane curvature and vehicle offset is defined in adv_find_lane_lines().

When the pipeline is used for finding lane lines in video frames, it keeps tracking the lane lines in previous frames and optimize its lane line searching based on the lane information in previous frames.
In case it fails to detect lane lines continuously in certain frames, it switches back to searching with sliding window.

In [ ]:
def adv_find_lane_lines(img, cam):
    '''
    Pipeline for finding lane lines in image/video frame.
    It identifies the lane lines and visualize the lane line, drivable area,
    lane curvature and vehicle offset from lane center.
    
    img: raw image/frame
    cam: Object of camera matrix and distortion coefficients    
    '''
    # 1. Undistort
    undist_img = cv2.undistort(img, cam.mtx, cam.dist, None, cam.mtx)
    # 2. Color & Gradient threshold
    color_binary, combined_binary = color_gradient_threshold(undist_img)
    # 3. Perspective Transform
    transformed, M = perspective_transform(combined_binary)
    
    result_img = img
    detected = False
    
    # 4. Search Lane line pixels and fit/draw 2nd degree polynomial line
    if (lane_line.last_detected == False) or (lane_line.best_fit_left is None):
        detected, left_fit, right_fit, sliding_win_img = search_sliding_window(transformed)
        if detected == True:
            fitted, left_fitx, right_fitx, ploty = draw_lane(transformed, left_fit, right_fit)
        else:
            if lane_line.best_fit_left is not None:
                left_fit = lane_line.best_fit_left
                right_fit = lane_line.best_fit_right
                fitted, left_fitx, right_fitx, ploty = draw_lane(transformed, left_fit, right_fit)
            else:
                return result_img
    else:
        detected, left_fit, right_fit = search_around_poly(transformed, lane_line.best_fit_left, lane_line.best_fit_right)
        if detected == True:
            fitted, left_fitx, right_fitx, ploty = draw_lane(transformed, left_fit, right_fit)
        else:
            left_fit = lane_line.best_fit_left
            right_fit = lane_line.best_fit_right
            fitted, left_fitx, right_fitx, ploty = draw_lane(transformed, left_fit, right_fit)
            
    # 5.1 Calculate curvate and vehicle position relative to lane center
    left_curverad, right_curverad = measure_curvature_real(left_fitx, right_fitx, ploty)
    
    # 5.2 Calculate vehicle position to the lane center
    car_offset, lane_width = cal_car_pos(img, left_fit, right_fit)
    print("Vehicle offset from lane center: %.2fm" % car_offset)

    # 6. Sanity Check
    ret = is_valid_lane(left_curverad, right_curverad, car_offset, lane_width)
    if (ret == False) or (detected == False):
        lane_line.update(None, None)
    else:
        lane_line.update(left_fit, right_fit)
        
    curvature = (left_curverad + right_curverad)/2
    print("Lane Curvature: %.2fm" % curvature)
    
    # 7. Convert back to original perspective  
    inv_img, Minv = perspective_transform(fitted, inverse=True)

    result_img = cv2.addWeighted(img, 1, inv_img, 0.4, 0)
    
    # 8. Show Curvature and car offset
    cv2.putText(result_img, 'Radius of Lane Curvature: {:d} m'.format(int(curvature)), (130, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color=(255,0,0), thickness=3)
  
    if car_offset < 0:
        cv2.putText(result_img, 'Vehicle offset from lane center: {:.2f} m (L)'.format(abs(car_offset)), (130, 170), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color=(255,0,0), thickness=3)
    else:
        cv2.putText(result_img, 'Vehicle offset from lane center: {:.2f} m (R)'.format(abs(car_offset)), (130, 170), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color=(255,0,0), thickness=3)
        
    return result_img


def process_video(image, cam):
    result = adv_find_lane_lines(image, cam)
    return result

# Lane Finding Test
## Lane Finding in Image

In [ ]:
print("Lane Finding in Images...")
test_images = glob.glob('./test_images/test*.jpg')
tmp_images = glob.glob('./test_images/straight_lines*.jpg')
test_images = test_images + tmp_images

rows = len(test_images)

dst_dir = './output_images/'
for image_file in test_images:
    lane_line = LaneLine()
    (path_filename, ext) = os.path.splitext(image_file)
    filename = os.path.basename(path_filename)
    result_image_file = dst_dir + filename + "_final_result" + ext
    img = mpimg.imread(image_file)
    result_img = adv_find_lane_lines(img, cam)
    
    mpimg.imsave(result_image_file, result_img)
    
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    ax1.imshow(img)
    ax1.set_title('Original', fontsize=22)
    ax2.imshow(result_img)
    ax2.set_title('Lane Lines Detection', fontsize=22)

print("Lane Finding in Images...Done!")

## Lane Finding in Video
### Project Video

In [ ]:
print("Processing normal video...")
lane_line = LaneLine()
video_output1 = 'output_images/project_video_output.mp4'
clip_video1 = VideoFileClip('project_video.mp4')
test_clip1 = clip_video1.fl_image(lambda image: process_video(image, cam))
test_clip1.write_videofile(video_output1, audio=False)

Play the resultant videos inline, or find the video in filesystem (in "output_images/"):

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output1))

### Challenge Video

In [ ]:
print("Processing challenge video...")
lane_line = LaneLine()
video_output2 = 'output_images/challenge_video_output.mp4'
clip_video2 = VideoFileClip('challenge_video.mp4')
test_clip2 = clip_video2.fl_image(lambda image: process_video(image, cam))
test_clip2.write_videofile(video_output2, audio=False)

Play the resultant videos inline, or find the video in filesystem (in "output_images/"):

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output2))

### Harder Challenge Video

In [ ]:
print("Processing harder challenge video...")
lane_line = LaneLine()
video_output3 = 'output_images/harder_challenge_video_output.mp4'
clip_video3 = VideoFileClip('harder_challenge_video.mp4')
test_clip3 = clip_video3.fl_image(lambda image: process_video(image, cam))
test_clip3.write_videofile(video_output3, audio=False)

Play the resultant videos inline, or find the video in filesystem (in "output_images/"):

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output3))